# Task 4 - AES mit Keygen
## 1. AES Encrypt/Decrypt (aus Task 3)

In [1]:
import re
import numpy as np

def read_sbox_file(filename):
    with open(filename, "r") as f:
        content = f.read()
        # Werte in einer Liste der Länge 256 voneinander isolieren
        vals = re.split(",\s", content)
        # Werte als Hexadezimalzahlen parsen
        int_vals = [int(val, 16) for val in vals]
        # Zu numpy Array konvertieren (ein Wert = 1 byte = np.uint8) und reshape
        return np.array(int_vals, dtype=np.uint8).reshape((16, 16))

sbox = read_sbox_file("SBox.txt")
assert sbox[0,0] == 0x63
sbox_inv = read_sbox_file("SBoxInvers.txt")
assert sbox_inv[0,0] == 0x52

def str_to_matrix(str):
    assert len(str) == 16
    return np.array([ord(c) for c in str], dtype=np.uint8).reshape((4,4)).transpose()

def matrix_to_str(mat):
    chars = [chr(c) for c in np.nditer(mat.transpose(), order="C")]
    return "".join(chars)

# Test
input_str = "Das ist ein Test"
mat = str_to_matrix(input_str)
out_str = matrix_to_str(mat)
assert input_str == out_str

def add_round_key(mat, key):
    return np.bitwise_xor(mat, key)

# Test
test_mat = np.array(mat, dtype=np.uint8)
test_key = str_to_matrix("1234567890123456")
test_mat_key = add_round_key(test_mat, test_key)
test_mat_key_inv = add_round_key(test_mat_key, test_key)
assert np.all(test_mat == test_mat_key_inv)

# Zugriff auf S-Box für einzelnes Byte
def get_sbox(val, inv):
    s_row = (val & 0xf0) >> 4
    s_col = val & 0xf
    return sbox_inv[s_row, s_col] if inv else sbox[s_row, s_col]

def sub_bytes(mat, inv=False):
    # appy get_sbox on every value in matrix
    f = np.vectorize(lambda val: get_sbox(val, inv))
    return f(mat)

# Test
test_mat = np.array(mat, dtype=np.uint8)
sub_mat = sub_bytes(test_mat)
subinv_mat = sub_bytes(sub_mat, inv=True)
assert np.all(test_mat == subinv_mat)

def shift_rows(mat, inv=False):
    if inv:
        # Zyklische Verschiebung nach rechts
        mat[1,:] = mat[1,[3,0,1,2]]
        mat[2,:] = mat[2,[2,3,0,1]]
        mat[3,:] = mat[3,[1,2,3,0]]
    else:
        # Zyklische Verschiebung nach links
        mat[1,:] = mat[1,[1,2,3,0]]
        mat[2,:] = mat[2,[2,3,0,1]]
        mat[3,:] = mat[3,[3,0,1,2]]
    return mat

# Test
test_mat = np.array(mat, dtype=np.uint8)
shifted_mat = shift_rows(test_mat)
shiftedinv_mat = shift_rows(shifted_mat, inv=True)
assert np.all(test_mat == shiftedinv_mat)

# gemaess Foliendefinition, multipliziert Polynom a(x) mit Polynom p(x) = x
def xtimes(a):
    t = a << 1
    if a & 0b10000000 != 0:
        t = np.bitwise_xor(t, 0x1b)
    return t

# Matrizen zur Multiplikation
mc_mat = np.array([
    [2, 3, 1, 1],
    [1, 2, 3, 1],
    [1, 1, 2, 3],
    [3, 1, 1, 2]], dtype=np.uint8)
mc_mat_inv = np.array([
    [0xe, 0xb, 0xd, 0x9],
    [0x9, 0xe, 0xb, 0xd],
    [0xd, 0x9, 0xe, 0xb],
    [0xb, 0xd, 0x9, 0xe]], dtype=np.uint8)

# Funktion zum Ausfuehren der Matrixmultiplikation auf einer einzelnen Spalte
def mix_single_col(in_col, inv):
    global mc_mat, mc_mat_inv
    mat = mc_mat_inv if inv else mc_mat
    out_col = np.zeros(in_col.shape, dtype=np.uint8)
    # Iteriere durch Matrix
    for row in range(4):
        for col in range(4):
            # Iteriere durch bits in mat[row, col]
            for bitpos in range(8):
                if mat[row, col] & (1 << bitpos) != 0:
                    # Multipliziere einstelliges Polynom (z.B. x^2 = x * x) mit Eingabepolynom
                    prod = in_col[col]
                    for _ in range(bitpos):
                        prod = xtimes(prod)
                    # Addiere das Produkt zum Ausgabepolynom
                    out_col[row] = np.bitwise_xor(out_col[row], prod)
    return out_col

def mix_columns(mat, inv=False):
    for col in range(4):
        mat[:,col] = mix_single_col(mat[:,col], inv)
    return mat

# Test
test_mat = np.array(mat, dtype=np.uint8)
mixed_test_mat = mix_columns(test_mat)
mixed_test_mat_inv = mix_columns(mixed_test_mat, inv=True)
assert np.all(test_mat == mixed_test_mat_inv)

def aes_encrypt(mat, keys, verbose=False):
    assert mat.shape == (4,4)
    assert keys.shape == (11, 4, 4)
    enc_mat = add_round_key(np.array(mat, dtype=np.uint8), keys[0])
    for i in range(1, 10):
        enc_mat = sub_bytes(enc_mat)
        enc_mat = shift_rows(enc_mat)
        enc_mat = mix_columns(enc_mat)
        enc_mat = add_round_key(enc_mat, keys[i])
        if verbose:
            print(i, matrix_to_str(enc_mat))
    enc_mat = sub_bytes(enc_mat)
    enc_mat = shift_rows(enc_mat)
    enc_mat = add_round_key(enc_mat, keys[10])
    return enc_mat

def aes_decrypt(mat, keys, verbose=False):
    assert mat.shape == (4,4)
    assert keys.shape == (11, 4, 4)
    # xor muss nicht umgekehrt werden (in x-or key = out <=> out x-or key = in)
    dec_mat = add_round_key(np.array(mat, dtype=np.uint8), keys[10])
    dec_mat = shift_rows(dec_mat, inv=True)
    dec_mat = sub_bytes(dec_mat, inv=True)
    for i in range(9, 0, -1):
        if verbose:
            print(i, matrix_to_str(enc_mat))
        dec_mat = add_round_key(dec_mat, keys[i])
        dec_mat = mix_columns(dec_mat, inv=True)
        dec_mat = shift_rows(dec_mat, inv=True)
        dec_mat = sub_bytes(dec_mat, inv=True)
    dec_mat = add_round_key(dec_mat, keys[0])
    return dec_mat

# Test
test_mat = np.array(mat, dtype=np.uint8)
keys = np.random.randint(0, 255, size=(11, 4, 4), dtype=np.uint8)
enc_test_mat = aes_encrypt(test_mat, keys)
dec_test_mat = aes_decrypt(enc_test_mat, keys)
assert np.all(dec_test_mat == test_mat)

## 2. Schlüsselgenerierung
Wir betrachten den Text- und Kryptoblock als 4x4 Matrix mit je einem Byte - es ist daher sinnvoll, jedes Wort auch als Vektor der Länge 4 mit je einem Byte zu betrachten.

In [2]:
# Falls word als einzelner int gegeben
def sub_word_alt(word):
    # Substituiere jedes Byte im Wort einzeln
    out = 0
    for i in range(4):
        b = (word >> (i * 8)) & 0xff
        s_b = get_sbox(b)
        out |= s_b << (i * 8)
    return out

# ab hier word als [b0, b1, b2, b3] gegeben
def sub_word(word):
    return sub_bytes(word)

def rot_word(word):
    return np.roll(word, shift=-1)

# Test
assert np.all(rot_word([0, 1, 2, 3]) == np.array([1, 2, 3, 0]))

def rcon(i):
    vec = np.zeros(4, dtype=np.uint8)
    vec[0] = 1
    for j in range(i - 1):
        vec[0] = xtimes(vec[0])
    return vec

# Test
rci = np.array([rcon(i) for i in range(1, 11)])
assert np.all(rci[:,0] == np.array([0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36]))
assert np.all(rci[:,1:] == 0)

In [4]:
def generate_round_keys(key):
    assert key.shape == (4,4)
    W = np.zeros((11, 4, 4), dtype=np.uint8)
    W[0] = key
    for i in range(4, 44):
        k = i // 4
        if i % 4 == 0:
            # W[i] = W[i-4] ^ rcon(i/4) ^ SubWord(RotWord(W[i-1]))
            W[k, :, 0] = W[k - 1, :, 0] ^ rcon(k) ^ sub_word(rot_word(W[k - 1, :, 3]))
        else:
            # W[i] = W[i-4] ^ W[i-1]
            W[k, :, i % 4] = W[k - 1, :, i % 4] ^ W[k, :, i % 4 - 1]
    return W

# Test
key_text = "EINSCHLUESSEL123"
print(generate_round_keys(str_to_matrix(key_text)))

[[[ 69  67  69  76]
  [ 73  72  83  49]
  [ 78  76  83  50]
  [ 83  85  69  51]]

 [[131 192 133 201]
  [106  34 113  64]
  [141 193 146 160]
  [122  47 106  89]]

 [[136  72 205   4]
  [138 168 217 153]
  [ 70 135  21 181]
  [167 136 226 187]]

 [[ 98  42 231 227]
  [ 95 247  46 183]
  [172  43  62 139]
  [ 85 221  63 132]]

 [[195 233  14 237]
  [ 98 149 187  12]
  [243 216 230 109]
  [ 68 153 166  34]]

 [[ 45 196 202  39]
  [ 94 203 112 124]
  [ 96 184  94  51]
  [ 17 136  46  12]]

 [[ 29 217  19  52]
  [157  86  38  90]
  [158  38 120  75]
  [221  85 123 119]]

 [[227  58  41  29]
  [ 46 120  94   4]
  [107  77  53 126]
  [197 144 235 156]]

 [[145 171 130 159]
  [221 165 251 255]
  [181 248 205 179]
  [ 97 241  26 134]]

 [[156  55 181  42]
  [176  21 238  17]
  [241   9 196 119]
  [186  75  81 215]]

 [[ 40  31 170 128]
  [ 69  80 190 175]
  [255 246  50  69]
  [ 95  20  69 146]]]


## 3. ECB Ver-/Entschlüsselung
Im ECB (Electronic Code Book Mode) wird jeder Block unabhängig voneinander mit dem gleichen Schlüssel verschlüsselt. Es ist die einfachste und zugleich unsicherste Betriebsart, da Klartextmuster erhalten bleiben.

In [17]:
# Aktualisiere str_to_matrix, sodass Nullen aufgefüllt werden
def str_to_matrix(str):
    # zero pad
    str += chr(0) * (16 - len(str))
    return np.array([ord(c) for c in str], dtype=np.uint8).reshape((4,4)).transpose()

In [22]:
# Funktion zum Ver- und Entschlüsseln beliebiger Zeichenketten mit ECB und AES
def ecb(text, key, decrypt=False):
    out = ""
    keys = generate_round_keys(key)
    for blocki in range(0, len(text), 16):
        block = str_to_matrix(text[blocki:(blocki+16)])
        out += matrix_to_str(aes_decrypt(block, keys) if decrypt else aes_encrypt(block, keys))
    return out

In [27]:
# Tests
sample_text = "Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet."
key = str_to_matrix("kryptologie_cool")
encoded = ecb(sample_text, key)
print("Encoded:", encoded)
decoded = ecb(encoded, key, decrypt=True)
print("Decoded:", decoded)
# Aufgefüllte Nullen am Ende werden nicht angezeigt und für den Assert separat entfernt
assert decoded.replace("\0", "") == sample_text

Encoded: (&fôCÓÃ¿0ëÍì9ò6àXÜÔ~iÀP	'Uóþ!á3ÊðUk-í0þÞ3ÃQË(±®¡çÄ0mA÷ Õ«é#]%¶Ïý²vØé¤RÿÌ­Â6«j¡eðE#)
â"ú§p;*b¦? Úùó¡â·_m<þzÁWßv` c§nP¿5B_üE_BP³·ÔuÕ?äÚRµ¹bA\ôÇ^-¹k­\BxwaejY"<!b4³o1Í»E'­ôÞææö§Ì 4AýWSû¼6/öõ|ò®Ð659<¾v4M	F)ôä{6±ò)
LÀÑ¾ßs­ÅQØ¦.1»ÓbªI¹¼Ö{W3Îmú(&s«Îä¹þRKRÕ&*ìföj\;nq¨Âäþ÷QÌ\+=iÔóYa®ûÕ¤}PÐ­IÃn&fHÖ[æåÞÉFJ6!A$©3¦õ%ÖP]	«hÜd³'EHÐäp$@à¥%`Û_Ø­x_¯¨½õ}^]7ñà}¥!ë8ØFÄMÏúÌ fb«8Ë.¥urëÝ§®²«BNÖ
Decoded: Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et e